In [46]:
!pwd

/Volumes/MTS800/tmp/hust/study_at_hust/semesters/2025_1/DS/hello_ngoc_linh/vietstock


In [47]:
!pip install beautifulsoup4 requests pandas numpy openpyxl openai python-dotenv selenium webdriver-manager httpx pydantic tqdm

In [48]:
import os 
import re
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import json
from datetime import datetime

In [49]:
LIMIT_CATEGORY = True
NUM_CATEGORIES = 5

# Maximum pages to crawl per category
MAX_PAGE_PER_CATEGORY = 5

# Date filtering option
FILTER_BY_DATE = False  # Set to False to disable date filtering

# Date range filter (inclusive)
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime(2024, 12, 31)

# Mapping of VN30 stock codes to their related keywords
KEYWORDS_MAP = {
    "ACB": ["ACB", "Ngân hàng ACB", "Ngân hàng TMCP Á Châu"],
    "BCM": ["BCM", "Becamex", "KCN Bình Dương", "khu công nghiệp Bình Dương", "VSIP", "Becamex IDC"],
    "BID": ["BIDV", "Ngân hàng Đầu tư và Phát triển Việt Nam"],
    "CTG": ["CTG", "VietinBank", "Ngân hàng Công Thương Việt Nam"],
    "DGC": ["DGC", "Hóa chất Đức Giang"],
    "FPT": ["FPT"],
    "GAS": ["PV GAS", "PV Gas", "Tổng Công ty Khí Việt Nam"],
    "GVR": ["GVR", "Tập đoàn Cao su", "Tập đoàn Công nghiệp Cao su Việt Nam"],
    "HDB": ["HDB", "HDBank", "Ngân hàng TMCP Phát triển Thành phố Hồ Chí Minh"],
    "HPG": ["HPG", "Hòa Phát"],
    "LPB": ["LPB", "LPBank", "LienVietPostBank", "Ngân hàng Bưu điện Liên Việt"],
    "MBB": ["MBB", "MBBank", "Ngân hàng Quân đội", "MB", "Ngân hàng TMCP Quân đội"],
    "MSN": ["MSN", "Masan", "WinCommerce"],
    "MWG": ["MWG", "Thế Giới Di Động", "Mobile World", "Bách Hóa Xanh", "BHX", "Điện Máy Xanh", "ĐMX", "TGDĐ"],
    "PLX": ["PLX", "Petrolimex", "Tập đoàn Xăng dầu Việt Nam"],
    "SAB": ["SAB", "Sabeco", "Tổng Công ty CP Bia - Rượu - Nước giải khát Sài Gòn"],
    "SHB": ["SHB", "Ngân hàng Thương mại Cổ phần Sài Gòn – Hà Nội", "Ngân hàng TMCP Sài Gòn Hà Nội"],
    "SSB": ["SSB", "Ngân hàng Thương mại Cổ phần Đông Nam Á", "Ngân hàng TMCP Đông Nam Á", "SeABank"],
    "SSI": ["SSI", "Chứng khoán SSI"],
    "STB": ["STB", "Sài Gòn Thương Tín", "Sacombank"],
    "TCB": ["TCB", "Techcombank", "Ngân hàng TMCP Kỹ Thương Việt Nam"],
    "TPB": ["TPB", "TPBank", "Ngân hàng Tiên Phong", "Ngân hàng TMCP Tiên Phong"],
    "VCB": ["VCB", "Vietcombank", "Ngân hàng TMCP Ngoại Thương Việt Nam", "Ngân hàng Ngoại thương"],
    "VHM": ["VHM", "Vinhomes"],
    "VIB": ["VIB", "Ngân hàng TMCP Quốc Tế Việt Nam", "Ngân hàng Quốc Tế"],
    "VIC": ["VIC", "Vingroup", "Công ty Cổ phần Tập đoàn Vingroup"],
    "VJC": ["VJC", "Vietjet Air", "Công ty Cổ phần Hàng không Vietjet", "máy bay Vietjet"],
    "VNM": ["VNM", "Vinamilk", "Công ty Cổ phần Sữa Việt Nam"],
    "VPB": ["VPB", "VPBank", "Ngân hàng TMCP Việt Nam Thịnh Vượng"],
    "VRE": ["VRE", "Vincom Retail", "Công ty Cổ phần Vincom Retail"]
}

In [50]:
def filter_articles_by_keywords(articles, keywords_map=KEYWORDS_MAP):
    # Filter articles containing VN30 stock keywords
    filtered = []
    
    for art in articles:
        # Convert title to lowercase for case-insensitive matching
        text = art["title"].lower()
        matched_codes = []
        
        # Check each stock code
        for code, keywords in keywords_map.items():
            for keyword in keywords:
                # Use word boundary regex to match full words only
                pattern = r'\b' + re.escape(keyword.lower()) + r'\b'
                if re.search(pattern, text):
                    matched_codes.append(code)
                    break
        
        # Keep articles with at least one matched stock code
        if matched_codes:
            art_copy = art.copy()
            art_copy["codes"] = list(set(matched_codes))  # Remove duplicates
            filtered.append(art_copy)
    
    return filtered

In [51]:
# HTTP headers to simulate a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'https://vietstock.vn',
    'Referer': 'https://vietstock.vn/tai-chinh.htm',
}

# Mapping of category URLs to their channel IDs
CATEGORY_CHANNEL_MAP = {
    'https://vietstock.vn/chung-khoan.htm': '144',
    'https://vietstock.vn/doanh-nghiep.htm': '733',
    'https://vietstock.vn/bat-dong-san.htm': '763',
    'https://vietstock.vn/tai-chinh.htm': '734',
    'https://vietstock.vn/hang-hoa.htm': '2',
    'https://vietstock.vn/kinh-te/vi-mo.htm': '761',
    'https://vietstock.vn/kinh-te/kinh-te-dau-tu.htm': '768',
    'https://vietstock.vn/the-gioi.htm': '736',
    'https://vietstock.vn/dong-duong.htm': '1317',
    'https://vietstock.vn/tai-chinh-ca-nhan.htm': '4259',
    'https://vietstock.vn/nhan-dinh-phan-tich.htm': '579',
    'https://vietstock.vn/san-giao-dich-tai-chinh.htm': '4645',
    'https://vietstock.vn/kinh-te.htm': '5307',
    'https://vietstock.vn/goc-nhin.htm': '4314',
}

def crawl_vietstock_category(url_category, channel_id, max_pages=3, output_file=None):
    # Update referer header for this category
    headers['Referer']=url_category
    session = requests.Session()
    articles_data = []
    
    # Step 1: Request the main page to get cookies
    print(f"Accessing main page: {url_category}")
    response = session.get(url_category, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to access main page (status: {response.status_code})")
        return articles_data

    soup_main = BeautifulSoup(response.text, 'html.parser')
    
    # Get authentication token if available
    token = ""
    token_tag = soup_main.find('input', {'name': '__RequestVerificationToken'})
    if token_tag:
        token = token_tag.get('value')
        print(f"Token obtained: {token[:20]}...")

    # Step 2: Loop through pages (pagination)
    api_url = "https://vietstock.vn/StartPage/ChannelContentPage"
    
    for page in range(1, max_pages + 1):
        print(f"  Crawling page {page}/{max_pages}...")
        
        # Prepare payload for API request
        payload = {
            'channelID': channel_id,
            'page': page,
            '__RequestVerificationToken': token
        }
        
        try:
            post_response = session.post(api_url, data=payload, headers=headers)
            
            if post_response.status_code == 200:
                soup_api = BeautifulSoup(post_response.text, 'html.parser')
                # Find all article titles
                articles = soup_api.find_all('h4', class_='channel-title') 
                
                if not articles:
                    articles = soup_api.select('h4 > a')

                # Check if page is empty (no more articles)
                if not articles:
                    print(f"  ⚠ No articles found on page {page}. Reached end of available pages.")
                    break

                page_articles_count = 0
                # Extract article data
                for article in articles:
                    link_tag = article.find('a') if article.name != 'a' else article
                    
                    if link_tag:
                        title = link_tag.get('title') or link_tag.text.strip()
                        href = link_tag.get('href')
                        
                        # Build full URL if relative
                        if href and not href.startswith('http'):
                            full_link = f"https://vietstock.vn{href}"
                        else:
                            full_link = href
                        
                        # Create article data dictionary
                        article_data = {
                            'title': title,
                            'link': full_link,
                        }
                        articles_data.append(article_data)
                        page_articles_count += 1
                        
                        # Append to output file
                        if output_file:
                            with open(output_file, 'a', encoding='utf-8') as f:
                                f.write(json.dumps(article_data, ensure_ascii=False) + '\n')
                
                print(f"  ✓ Found {page_articles_count} articles on page {page}")
            else:
                print(f"  ✗ Error calling API page {page}: {post_response.status_code}")
                break
                
        except Exception as e:
            print(f"  ✗ Error occurred: {e}")
            break
            
        # Delay between requests
        time.sleep(2)
    
    print(f"Completed category: {len(articles_data)} total articles")
    return articles_data

# Output filename
output_filename = f'vietstock_articles.jsonl'

print(f"\n{'='*60}")
print(f"Starting crawl process")
print(f"Max pages per category: {MAX_PAGE_PER_CATEGORY}")
print(f"Total categories: {len(CATEGORY_CHANNEL_MAP)}")
print(f"{'='*60}\n")

# Crawl all categories
all_articles = []
cnt = 0
for idx, (category_url, channel_id) in enumerate(CATEGORY_CHANNEL_MAP.items(), 1):
    print(f"\n[{idx}/{len(CATEGORY_CHANNEL_MAP)}] Category: {category_url}")
    print(f"Channel ID: {channel_id}")
    print("-" * 60)
    
    articles = crawl_vietstock_category(category_url, channel_id, max_pages=MAX_PAGE_PER_CATEGORY, output_file=output_filename)
    all_articles.extend(articles)
    
    print(f"Total articles so far: {len(all_articles)}")
    time.sleep(3)  # Delay between categories
    
    cnt += 1
    if LIMIT_CATEGORY:
        if cnt >= NUM_CATEGORIES:
            print(f"\n⚠ Category limit reached ({NUM_CATEGORIES}). Stopping...")
            break

print(f"\n{'='*60}")
print(f"✓ Crawling completed!")
print(f"Total articles saved: {len(all_articles)}")
print(f"Output file: {output_filename}")
print(f"{'='*60}")


Starting crawl process
Max pages per category: 5
Total categories: 14


[1/14] Category: https://vietstock.vn/chung-khoan.htm
Channel ID: 144
------------------------------------------------------------
Accessing main page: https://vietstock.vn/chung-khoan.htm
Token obtained: -k6MVafOu7tzemjgDz6h...
  Crawling page 1/5...
Token obtained: -k6MVafOu7tzemjgDz6h...
  Crawling page 1/5...
  ✓ Found 10 articles on page 1
  ✓ Found 10 articles on page 1
  Crawling page 2/5...
  Crawling page 2/5...
  ✓ Found 10 articles on page 2
  ✓ Found 10 articles on page 2
  Crawling page 3/5...
  Crawling page 3/5...
  ✓ Found 10 articles on page 3
  ✓ Found 10 articles on page 3
  Crawling page 4/5...
  Crawling page 4/5...
  ✓ Found 10 articles on page 4
  ✓ Found 10 articles on page 4
  Crawling page 5/5...
  Crawling page 5/5...
  ✓ Found 10 articles on page 5
  ✓ Found 10 articles on page 5
Completed category: 50 total articles
Total articles so far: 50
Completed category: 50 total articles
Total 

In [52]:
def extract_article_metadata(article_url):
    """
    Extract publication date and summary from article page's meta tags.
    Returns tuple (date, summary) in format (YYYY-MM-DD, summary_text) or (None, None) if not found.
    """
    try:
        # Use proper browser headers to avoid 403 errors
        metadata_headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://vietstock.vn/',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0'
        }
        
        response = requests.get(article_url, headers=metadata_headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract date from article:published_time meta tag
        date_part = None
        meta_date_tag = soup.find('meta', property='article:published_time')
        if meta_date_tag and meta_date_tag.get('content'):
            datetime_str = meta_date_tag['content']
            date_part = datetime_str.split('T')[0]
        
        # Extract summary from description meta tag
        summary = "N/A"
        meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
        if meta_desc_tag and meta_desc_tag.get('content'):
            summary = meta_desc_tag['content'].strip()
        
        return date_part, summary
        
    except Exception as e:
        print(f"  ✗ Error extracting metadata: {e}")
        return None, "N/A"


def is_date_in_range(date_str, start_date=START_DATE, end_date=END_DATE):
    """
    Check if date string is within the specified range.
    """
    if not date_str or date_str == 'N/A':
        return False
    try:
        article_date = datetime.strptime(date_str, '%Y-%m-%d')
        return start_date <= article_date <= end_date
    except:
        return False


def filter_and_group_by_codes(articles_list, keywords_map=KEYWORDS_MAP):
    """
    Filter articles by VN30 keywords and group by stock codes.
    Uses word boundaries to match full keywords only.
    Returns a list of dictionaries with stock code and associated articles.
    """
    # Dictionary to store articles for each code (using link as key to avoid duplicates)
    code_articles = {code: {} for code in keywords_map.keys()}
    
    for article in articles_list:
        # Combine title and summary for keyword matching
        text = (article.get("title", "") + " " + article.get("summary", "")).lower()
        matched_codes = []
        
        # Check each stock code
        for code, keywords in keywords_map.items():
            for keyword in keywords:
                # Use word boundary regex to match full words only
                pattern = r'\b' + re.escape(keyword.lower()) + r'\b'
                if re.search(pattern, text):
                    matched_codes.append(code)
                    break
        
        # Add article to each matched code's dictionary (using link as key to prevent duplicates)
        article_link = article.get('link', '')
        for code in matched_codes:
            if article_link and article_link not in code_articles[code]:
                code_articles[code][article_link] = {
                    'Stock_Code': code,
                    'Title': article.get('title', ''),
                    'Link': article_link,
                    'Summary': article.get('summary', 'N/A')
                }
    
    # Flatten into single list, sorted by stock code
    result = []
    for code in sorted(code_articles.keys()):
        result.extend(code_articles[code].values())
    
    return result, {code: list(articles.values()) for code, articles in code_articles.items()}


# Read all articles from JSONL file
all_articles_list = []

with open(output_filename, 'r', encoding='utf-8') as f:
    for line in f:
        article = json.loads(line.strip())
        all_articles_list.append(article)

print(f"\n{'='*60}")
print("Filtering and grouping articles by VN30 stock codes...")
print(f"{'='*60}\n")

print(f"Total articles before filtering: {len(all_articles_list)}")

# Filter and group by codes
grouped_articles, code_dict = filter_and_group_by_codes(all_articles_list, keywords_map=KEYWORDS_MAP)

print(f"Articles matching VN30 keywords: {len(grouped_articles)}")

# Extract publication dates and summaries for filtered articles
if grouped_articles:
    print(f"\n{'='*60}")
    print("Extracting dates and summaries from article pages...")
    print(f"{'='*60}\n")
    
    for idx, article in enumerate(grouped_articles, 1):
        print(f"[{idx}/{len(grouped_articles)}] Extracting metadata for: {article['Title'][:50]}...")
        pub_date, summary = extract_article_metadata(article['Link'])
        article['Date'] = pub_date if pub_date else 'N/A'
        article['Summary'] = summary
        time.sleep(0.5)  # Delay between requests
    
    print("\n✓ Metadata extraction completed")
    
    # Filter by date range
    if FILTER_BY_DATE:
        print(f"\n{'='*60}")
        print(f"Filtering articles by date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
        print(f"{'='*60}\n")
        
        articles_before_date_filter = len(grouped_articles)
        grouped_articles = [art for art in grouped_articles if is_date_in_range(art['Date'])]
        
        print(f"Articles before date filter: {articles_before_date_filter}")
        print(f"Articles after date filter: {len(grouped_articles)}")
    else:
        print(f"\n⚠ Date filtering is disabled (FILTER_BY_DATE=False)")

# Save to single CSV file
output_csv_filename = "titles_vn30_vietstock.csv"
if grouped_articles:
    df_output = pd.DataFrame(grouped_articles)
    # Reorder columns to have Date after Stock_Code and include Summary
    cols = ['Stock_Code', 'Date', 'Title', 'Link', 'Summary']
    df_output = df_output[cols]
    df_output.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')
    print(f"✓ Saved to '{output_csv_filename}'")
    
    # Recalculate statistics from date-filtered articles
    code_stats_filtered = {}
    for article in grouped_articles:
        code = article['Stock_Code']
        code_stats_filtered[code] = code_stats_filtered.get(code, 0) + 1
    
    # Print statistics by stock code
    print(f"\n{'='*60}")
    print("Statistics by stock code (after date filtering):")
    print(f"{'='*60}")
    for code in sorted(code_stats_filtered.keys()):
        count = code_stats_filtered[code]
        if count > 0:
            print(f"{code}: {count} articles")
    
    # Display sample
    print(f"\n{'='*60}")
    print("Sample articles (first 5):")
    print(f"{'='*60}")
    for idx, row in df_output.head(5).iterrows():
        print(f"\n[{row['Stock_Code']}] {row['Date']} - {row['Title']}")
        print(f"Link: {row['Link']}")
        print(f"Summary: {row['Summary'][:100]}..." if len(str(row['Summary'])) > 100 else f"Summary: {row['Summary']}")
else:
    print("No articles matched VN30 keywords or date range!")


Filtering and grouping articles by VN30 stock codes...

Total articles before filtering: 250
Articles matching VN30 keywords: 18

Extracting dates and summaries from article pages...

[1/18] Extracting metadata for: VietinBank: Mục tiêu lợi nhuận trước thuế riêng lẻ...
[2/18] Extracting metadata for: HDBank hợp tác với Trung tâm RAR - Bộ Công an: Tíc...
[2/18] Extracting metadata for: HDBank hợp tác với Trung tâm RAR - Bộ Công an: Tíc...
[3/18] Extracting metadata for: HDBank được Visa vinh danh 2 giải thưởng quan trọn...
[3/18] Extracting metadata for: HDBank được Visa vinh danh 2 giải thưởng quan trọn...
[4/18] Extracting metadata for: FMO và HDBank ký Biên bản ghi nhớ hợp tác, triển k...
[4/18] Extracting metadata for: FMO và HDBank ký Biên bản ghi nhớ hợp tác, triển k...
[5/18] Extracting metadata for: Ấn Độ áp thuế chống bán phá giá lên thép HRC Việt ...
[5/18] Extracting metadata for: Ấn Độ áp thuế chống bán phá giá lên thép HRC Việt ...
[6/18] Extracting metadata for: MB bắt ta

In [53]:
os.remove("./vietstock_articles.jsonl")